In [1]:
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
import sqlite3

In [ ]:
DB_FILE="LibraryDB.db"
def init_db():
    db=sqlite3.connect(DB_FILE)
    cursor=db.cursor()
    # Create a table called Books with foru fields
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS Books(
            isbn text PRIMARY KEY,
            title text NOT NULL,
            author text NOT NULL,
            year integer);
    """)

    db.commit()
    db.close()
    print("Database initialized successfully!")
    
def save_book(isbn,title,author,year):
    db=sqlite3.connect(DB_FILE)
    cursor=db.cursor()
    try:
        # Insert data into the table
        cursor.execute("INSERT INTO Books(isbn,title,author,year)VALUES(?,?,?,?)",(isbn,title,author,year))
        db.commit()
        return True,"Saved"
    except sqlite3.IntegrityError as e:
        return False,str(e)
    finally:
        db.close()

def save_button_clicked():
    isbn = isbn_entry.get()
    title = title_entry.get()
    author = author_entry.get()
    year = year_entry.get()
    
    if not isbn or not title or not author or not year:
        messagebox.showwarning("Warning", "All fields are required!")
        return

    success, msg = save_book(isbn, title, author, year)
    if success:
        messagebox.showinfo("Success", msg)
        clear_fields()
    else:
        messagebox.showerror("Error", msg)
    
    current_year = datetime.now().year
    try:
        year_int = int(year)
        if year_int < 1450 or year_int > current_year:
            messagebox.showwarning("Invalid year of publication!", f"Year must be between 1450 and {current_year}.")
            return
    except ValueError:
        messagebox.showwarning("Invalid Year", "Year must be a number.")
        return
     
        
def clear_fields():
    isbn_entry.delete(0, tk.END)
    title_entry.delete(0, tk.END)
    author_entry.delete(0, tk.END)
    year_entry.delete(0, tk.END)
    books_list.delete(*books_list.get_children())

def fetch_books():
    db = sqlite3.connect(DB_FILE)
    cursor = db.cursor()
    cursor.execute("SELECT * FROM Books")
    rows = cursor.fetchall() #type(rows) list
    db.close()
    return rows
# 4. If a user clicks the “Display Books” button, the system should display all book details saved in the database. [3 marks]
def display_books():
    for row in fetch_books():
        books_list.insert("", tk.END, values=row)

##### When a SQL query returns multiple results, they come back as a **list of tuples**

Example result from `fetchall()`:

```python
[
    ("9781234567890", "Python Basics", "John Smith", 2021),
    ("9780987654321", "AI Fundamentals", "Mary Johnson", 2022)
]


Both represent a table of records
The only difference is the syntax — because the data structure (list vs DataFrame) is different.

That’s why the way you write code to access or iterate over them is different。

So conceptually:

“Iterating through a list of tuples from SQL”
≈
“Iterating through rows in a DataFrame”

They both mean: “go through each record in a table, one by one.”

In [ ]:
def simulated_chatgpt(query):
    db=sqlite3.connect(DB_FILE)
    cursor=db.cursor()
    
    query=query.lower()
    
    if 'recommend me a book by' in query:
        author_name = query.replace("recommend me a book by ", "").strip()
        # cursor.execute(select*from db where author==author_name)
        cursor.execute("SELECT title FROM Books WHERE LOWER(author)=?", (author_name,))
        books=cursor.fetchall()
        db.close()

In [3]:
root=tk.Tk()
root.title("Library Management System")

ttk.Label(root,text="ISBN").grid(row=0,column=0)
isbn_entry = ttk.Entry(root, width=20); isbn_entry.grid(row=0, column=1)
ttk.Label(root, text="Title:").grid(row=1, column=0)
title_entry = ttk.Entry(root, width=20); title_entry.grid(row=1, column=1)
ttk.Label(root, text="Author:").grid(row=2, column=0)
author_entry = ttk.Entry(root, width=20); author_entry.grid(row=2, column=1)
ttk.Label(root, text="Year:").grid(row=3, column=0)
year_entry = ttk.Entry(root, width=20); year_entry.grid(row=3, column=1)

ttk.Button(root, text="Save",command=save_button_clicked).grid(row=4, column=0)
ttk.Button(root, text="Clear",command=clear_fields).grid(row=4, column=1)
ttk.Button(root, text="Display Books",command=display_books).grid(row=5, column=0)
ttk.Label(root, text="Display Area:").grid(row=6, column=0)

askLibrarian_entry = ttk.Entry(root, width=40); askLibrarian_entry.grid(row=8, column=0)
ttk.Label(root,text="Ask Librarian").grid(row=9,column=0)

columns = ("ISBN", "Title", "Author", "Year")
books_list = ttk.Treeview(root, columns=columns, show="headings")
for col in columns:
    books_list.heading(col, text=col)
books_list.grid(row=7, column=0, columnspan=4, padx=5, pady=10)

In [4]:
#if __name__ == "__main__":
init_db()
root.mainloop()

Database initialized successfully!
